In [ ]:
import numpy as np
import math
import random

In [ ]:
def binary(g,p,t):
    left = 0
    right = t-1
    
    while(left != right):
        middle = int((left + right)/2)
        if g > p[middle]:
            left = middle + 1
        else:
            right = middle
    
    return left

def RRkanon(k,g,s): #k-anonymization before randomized response
    gg = np.zeros(len(g))
    p = np.zeros(len(s))
    m = 0
    t = 0
    for i in range(len(s)):
        m += s[i]
        if m >= k:
            p[t] = i; t += 1; m = 0
    
    if m < k:
        p[t-1] = len(s)-1
    
    rep = np.zeros(t)
    r = np.zeros(t)
    
    rep[0] = int(p[0]/2)
    r[0] = p[0]
    for i in range(1,t):
        rep[i] = int((p[i-1]+p[i])/2)
        r[i] = p[i]-p[i-1]
    
    for i in range(len(g)):
        gg[i] = binary(g[i],p,t)
    
    return gg,t,rep,r

def kkanon(k,g,s,rs): #k-anonymization after randomized response
    gg = np.zeros(len(g))
    p = np.zeros(len(s))
    m = 0
    t = 0
    for i in range(len(s)):
        m += s[i]
        if m >= k:
            p[t] = i; t += 1; m = 0
    
    if m < k:
        p[t-1] = len(s)-1
    
    rep = np.zeros(t)
    r = np.zeros(t)
    ri = 0; j = 0
    
    for i in range(len(rs)):
        if rs[i] < p[j]:
            ri += 1
        else:
            ri += 1; r[j] = ri; j += 1; ri = 0
    
    rep[0] = int(p[0]/2)
    for i in range(1,t):
        rep[i] = int((p[i-1]+p[i])/2)
    
    for i in range(len(g)):
        gg[i] = binary(rs[int(g[i])],p,t)
        
    return gg,t,rep,r

In [ ]:
def RR(epsilon,g,m): #randomized response
    gRR = np.zeros(len(g))
    
    for i in range(len(g)):
        rand = random.random()
        if rand >= m/(math.exp(epsilon)+m-1):
            gRR[i] = g[i]
        else:
            gRR[i] = int(rand*(math.exp(epsilon)+m-1))
            
    return gRR

def RandomizedAnonymization(data,k,epsilon): #randomized anonymization
    g = np.zeros(len(data))
    for i in range(len(data)):
        g[i] = #score calculation
        
    s = np.zeros(#number of possible scores)
    for i in range(len(g)):
        s[int(g[i])] += 1
    
    kk = int(k/10)
    
    gkk,tt,rep,rr = RRkanon(kk,g,s)
    gkkRR = RR(epsilon,gkk,tt)
    
    rs = #recompute representative values
    
    ss = np.zeros(#number of possible scores)
    for i in range(len(gkkRR)):
        ss[int(rs[int(gkkRR[i])])] += 1
    
    gkkRRk,t,rep,r = kkanon(k,gkkRR,ss,rs)
    
    return gkkRRk,t,rep,r

In [ ]:
def RARecoverDis(RRDis, epsilon, r, n, m): #reconstruction using inverse matrix
    recoveredDis = np.zeros(m)
    x = np.sum(r)
    for i in range(m):
        recoveredDis[i] = round(((math.exp(epsilon)+x-r[i]-1)/(math.exp(epsilon)-1))*RRDis[i] - (r[i]*((n-RRDis[i])/(math.exp(epsilon)-1))))
        if recoveredDis[i] < 0:
            recoveredDis[i] = 0
    recoveredDis *= (np.sum(RRDis)/np.sum(recoveredDis))
        
    return recoveredDis

In [ ]:
def RAPrx(x,r,i,j,epsilon,m):
    X = np.sum(r)
    if i == j:
        if x == 1:
            return (math.exp(epsilon)+r[i]-1)/(math.exp(epsilon)+X-1)
        else:
            return (X-r[i])/(math.exp(epsilon)+X-1)
    else:
        if x == 1:
            return r[i]/(math.exp(epsilon)+X-1)
        else:
            return (math.exp(epsilon)+X-r[i]-1)/(math.exp(epsilon)+X-1)

def RAEM(RRDis, epsilon, r, n, m): #reconstruction using EM algorithm
    recoveredDis = np.zeros(m)
    x = np.zeros((n,m))
    for j in range(int(RRDis[0])):
        x[j][0] = 1
    s = RRDis[0]
    for i in range(1,m-1):
        for j in range(int(s),int(s+RRDis[i])):
            x[j][i] = 1
        s += RRDis[i]
    for j in range(int(s),n):
        x[j][m-1] = 1
    
    theta = (1/m) * np.ones(m)
    p = np.zeros((m,n))
    Theta = np.zeros(m)
    
    while(1):
        # E step
        for j in range(n):
            for i in range(m):
                top = RAPrx(x[j][i],r,i,i,epsilon,m) * theta[i]
                bottom = 0
                for k in range(m):
                    bottom += RAPrx(x[j][i],r,i,k,epsilon,m) * theta[k]
                p[i][j] = top/bottom
                
        # M step
        err = 0
        for i in range(m):
            Theta[i] = np.mean(p[i])
            err += math.fabs(Theta[i]-theta[i])
        
        if err < #delta:
            break
        else:
            for i in range(m):
                theta[i] = Theta[i]
    
    return n*Theta